# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
import scipy.sparse as sp
from tqdm import tqdm
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

## Загрузка и предобработка данных

Загрузим данные и проведем предобраотку данных как на семинаре.

In [2]:
!wget -q -N https://www.dropbox.com/s/z8syrl5trawxs0n/articles.zip?dl=0 -O articles.zip
!unzip -o -q articles.zip

In [3]:
articles_df = pd.read_csv('articles/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [4]:
interactions_df = pd.read_csv('articles/users_interactions.csv')
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [5]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [6]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [7]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())
print('# users:', len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions:',len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [8]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [9]:
print('# interactions before:', interactions_df.shape)
print('# interactions after:', interactions_from_selected_users_df.shape)

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [10]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [11]:
split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [12]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), 'true_test'] = [
    list() for x in range(len(interactions.loc[pd.isnull(interactions.true_test), 'true_test']))]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


## Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

## Задание 1. (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [14]:
interactions_train_df.head(2)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892


In [15]:
interactions_test_df.head(2)

,personId,contentId,eventStrength,last_timestamp
1,-1007001694607905623,-6623581327558800021,1.0,1487240080
3,-1007001694607905623,1469580151036142903,1.0,1487240062


In [67]:
# Ваш код здесь
def create_sparse_matrix(df, n_users, n_items):
    row = df['personId'].astype('category').cat.codes
    col = df['contentId'].astype('category').cat.codes
    data = df['eventStrength'].astype(float).to_numpy()
    return sp.coo_matrix((data, (row, col)), shape=(n_users,n_items))

n_users=len(np.unique(interactions_train_df['personId']))
n_items=len(np.unique(interactions_train_df['contentId']))    

data_train = create_sparse_matrix(interactions_train_df, n_users, n_items)
data_test = create_sparse_matrix(interactions_test_df, n_users, n_items)

In [68]:
data_train

<1112x2366 sparse matrix of type '<class 'numpy.float64'>'
	with 29329 stored elements in COOrdinate format>

In [69]:
data_test

<1112x2366 sparse matrix of type '<class 'numpy.float64'>'
	with 9777 stored elements in COOrdinate format>

## Задание 2. (1 балл)

Обучите модель LightFM с `loss='warp'` и посчитайте *precision@10* на тесте.

In [94]:
model = LightFM(loss='warp', k=10)

In [95]:
model.fit(data_train,epochs=100)

In [97]:
precision_at_k(model, data_test, k=10).mean()

0.007230143

## Задание 3. (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss='warp'` и посчитайте precision@10 на тесте.

In [104]:
vectorizer = TfidfVectorizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [117]:
def tfidf_feat_create(articles_df):
    text_df = articles_df[['contentId', 'text']].drop_duplicates(ignore_index=True)
    text = [' '.join(word) for word in text_df.text.apply(word_tokenize).tolist()]
    vectorizer = TfidfVectorizer()
    feat = vectorizer.fit_transform(text)
    return feat

In [118]:
feat = tfidf_feat_create(articles_df)

In [114]:
model_tfidf = LightFM(loss='warp', random_state=7, k=10)
model_tfidf.fit(data_train, item_features=feat)

In [116]:
precision_at_k(model_tfidf, data_test, k=10, item_features=feat).mean()

0.0034623218

## Задание 4. (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [ ]:
def lemmatize_and_stopwords(list_of_words):
    list_of_words = [lemmatizer.lemmatize(word) for word in list_of_words]
    list_of_words = [word for word in list_of_words if word not in stopwords.words('english')]
    return list_of_words


def new_feat_create(articles_df):
    articles_df['text_prep'] = articles_df['text'].str.lower()
    articles_df['text_prep'] = articles_df['text_prep'].apply(word_tokenize)
    lemmatizer = WordNetLemmatizer()
    articles_df['text_prep'] = articles_df.text_prep.apply(lambda x: lemmatize_and_stopwords(x))
    text = [' '.join(word) for word in articles_df.text_prep.tolist()]
    vectorizer = TfidfVectorizer()
    feat = vectorizer.fit_transform(text)
    return feat

In [129]:
feat = new_feat_create(articles_df)

In [138]:
model_tfidf_new = LightFM(loss='warp', random_state=7, k=10)
model_tfidf_new.fit(data_train, item_features=feat)

In [139]:
precision_at_k(model_tfidf_new, data_test, k=10, item_features=feat).mean()

0.005702648

Улучшилось ли качество предсказания? 
#### Ответ: очевидно улучшилось

## Задание 5. (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [169]:
no_components_array = np.arange(10, 80, 20)
losses_types = ['logistic', 'bpr', 'warp']
lr_array = np.arange(0.05, 0.9, 0.15)

In [177]:
result = []
for n_component in no_components_array:
    for loss_type in losses_types:
        for lr in lr_array:
            model_find = LightFM(loss=loss_type, random_state=7, k=10, no_components=n_component, learning_rate=lr)
            model_find.fit(data_train, item_features=feat)
            prec = precision_at_k(model_find, data_test, k=10, item_features=feat).mean()
            result.append([n_component, loss_type, lr, prec])

In [178]:
params = pd.DataFrame(result, columns=['no_components', 'loss_type', 'learning_rate', 'precision@10'])

In [179]:
params.sort_values(by='precision@10', ascending=False)

,no_components,loss_type,learning_rate,precision@10
8,10,bpr,0.35,0.013340
44,50,bpr,0.35,0.012016
66,70,warp,0.05,0.010183
62,70,bpr,0.35,0.009980
61,70,bpr,0.20,0.008350
...,...,...,...,...
51,50,warp,0.50,0.002648
7,10,bpr,0.20,0.002546
43,50,bpr,0.20,0.002444
17,10,warp,0.80,0.001731


#### Лучшим набором гиперпараметров является no_components = 10, loss_type = bpr и learning_rate = 0.35. При этих параметрах 	precision@10 = 0.013340